In [1]:
import os
import pandas as pd
from collections import Counter
import numpy as np
import random
import json

#supernoder files:

#вариант, который считает РАЗНЫЕ типы мотивов одного и того же размера
from SuperNoder_diff_types.manager import Manager as Manager_types 


#вариант, который считает все мотивы одного размера вместе
from SuperNoder.manager import Manager as Manager

#!pip install ipyparallel

**First of all will convert every dataset to a txt file with nodes as numbers**

In [2]:
#Let's choose graphs randomly. k - number of graphs
for root, dirs, files in os.walk(r"C:\Users\anpolol\Desktop\DifusionPredictionUsingMotifs\Data\reply_networks"):
    datasets_names = random.choices(files,k=128)
    
#in every graph there are 11 networks, all_nets consists of all nets  
all_nets = dict()
for dataset in datasets_names:
    if dataset not in all_nets:
        all_nets[dataset] = []
        month_nets = json.load(open(r"C:\Users\anpolol\Desktop\DifusionPredictionUsingMotifs\Data\reply_networks\\"+str(dataset)))
        all_nets[dataset]=all_nets[dataset] + month_nets
        
#Create txt files from json
path=r'Data/'
for dataset in all_nets:
    name = dataset.split('.')
    for i,net in enumerate(all_nets[dataset]):
        
        #Create nodes file
        t=list(net.values())
        nodes = (Counter([item for sublist in t for item in sublist] + list(net.keys()))) 
        map_nodes=dict(zip( nodes , list(range(len(nodes))))) #нумеруем ники
        if not os.path.exists(path+name[0]+"_"+str(i)+"_nodes"+".txt"):
            file_nodes = open(path+name[0]+"_"+str(i)+"_nodes"+".txt", "w+")
            for j in range(len(nodes)):
                file_nodes.write(str(j)+' A'+'\n')
            file_nodes.close()
        
        #Create edges files
        if not os.path.exists(path+name[0]+"_"+str(i)+".txt"):
            file= open(path+name[0]+"_"+str(i)+".txt", "w+") 
            for node in net:
                for neigh in net[node]:
                    file.write(str(map_nodes[node])+' '+str(map_nodes[neigh])+'\n') #first node replied to the second    
            file.close()

**Some statistics for graphs**:
- число узлов 
- плотность
- к-т кластеризации (общий, для графа)
- ассортативность (по степени)

In [ ]:
#построение графа из файла с ребрами
print('Number of nodes (NN), Clustering coefficient (CC), Density(D), Degree assortativity coefficient(DAC) \n')
for dataset in all_nets:
    print('DATASET: ', dataset,'\n')
    name = dataset.split('.')
    number_of_nodes = []
    clustering_coef=[]
    density=[]
    degree_assort=[]
    for i,net in enumerate(all_nets[dataset]):
        g = nx.read_edgelist(path+name[0]+"_"+str(i)+".txt",delimiter=' ', create_using=nx.DiGraph(), nodetype = int)
        #число узлов 
        number_of_nodes.append(g.number_of_nodes())
        #коэффициенты кластеризации ищутся следующим образом:
        clustering_coef.append(np.mean(list(nx.clustering(g).values())))
        #плотность
        density.append(nx.density(g))
        #ассортативность
        degree_assort.append(nx.degree_assortativity_coefficient(g))
    log = 'NN: {:.4f}, CC: {:.4f}, D: {:.4f}, DAC: {:.4f}'
    print('mean: ', log.format(np.mean(np.array(number_of_nodes)) ,np.mean(np.array(clustering_coef)) ,np.mean(np.array(density)),np.mean(np.array(degree_assort))),'\n')
    print('std: ',log.format(np.std(np.array(number_of_nodes)) ,np.std(np.array(clustering_coef)) ,np.std(np.array(density)),np.std(np.array(degree_assort))),'\n')

# **Motif counter - SuperNoder** 

**arguments for SuperNoder:**

-n,  --nodes-file <filename> MANDATORY The list of nodes. Node id and label for each row separated by a space\
-e,  --edges-file <filename> MANDATORY The list of edges. One edge for each row.\
-m,  --method <method> OPTIONAL The heuristic to use in order to maximize motifs. DEFAULT: h1 \
-tn, --type-of-network <type> OPTIONAL 	The type of network. It can be chosen from [direct, undirect]. DEFAULT: undirect\ 
-th, --threshold <threshold> OPTIONAL The threshold to hold over-represented motifs.\
-ms, --motif-size <size> OPTIONAL The size of motifs. It must be greater or equal to 3. DEFAULT: 3\
-h1tr, --h1-times-repetition <times> OPTIONAL \tThe number of repetition of h1. DEFAULT: 1\
-ss, --samples-size <sample_size> OPTIONAL The size of samples for heuristics h4 and h5. DEFAULT: 100\

# you should "ipcluster start -n 4" on your local machine
**instead of 4 you should write the number of engines on your computer**

In [ ]:
import ipyparallel as ipp
c = ipp.Client()
dview = c[:]

In [ ]:
#counting every netin a graph separately
args_e=[]
for dataset in all_nets:
    name = dataset.split('.')
    for i,net in enumerate(all_nets[dataset]):
        args_e.append(path+name[0]+"_"+str(i)+".txt")

In [ ]:
def func(arg_e):
    from SuperNoder_diff_types.manager import Manager as Manager_types 
    arg_tn='direct'
    arg_th = '1' 
    arg_m = 'h1'
    arg_ss = '100'
    arg_h1tr = 1 
    name=arg_e.split('.')
    arg_n = name[0]+"_nodes"+".txt"
    distributions = {}
    distributions_disjoint = {}
    
    for arg_ms in range(3,9):
        argv=[' ','-e',arg_e, '-n',arg_n, '-tn',arg_tn,'-th',arg_th,'-ms',arg_ms,'-m',arg_m,\
      '-h1tr',arg_h1tr,'-ss',arg_ss]
        m = Manager_types(argv)
        distribution_disjoint, distribution_f1 = m.run()
        distributions = dict(list(distributions.items()) + list(distribution_f1.items()))
        distributions_disjoint = dict(list(distributions_disjoint.items()) + list(distribution_disjoint.items()))
    return  arg_e,distributions_disjoint,distributions#первое значение словаря - тип мотива или размер мотива, второе значение - количество таких мотивов в графе

In [ ]:
# here is a parellelization
res = dview.map(func,args_e).get()

In [ ]:
#distributions of motifs for every net:
Motif_f1 = dict()
Motif_f3 = dict()
for (name,f3,f1) in res:
    Motif_f3[name] = f3
    Motif_f1[name] = f1

**составляем матрицу входных данных таким образом, чтоб она была одной и той же длины для любого датасета**

In [ ]:
names_of_all_motifs=[] #list of all motif types in all datasets
for dataset in Motif_f3:
    for name_of_motif in Motif_f3[dataset]:
        if name_of_motif not in names_of_all_motifs:
            names_of_all_motifs.append(str(name_of_motif))
names_of_all_motifs=sorted(names_of_all_motifs)

In [ ]:
X = np.zeros((len(args_e),len(names_of_all_motifs)))
for i,d in enumerate(args_e):
    for k,m in enumerate(names_of_all_motifs):
        if m in Motif_f3[d]:
            X[i][k] = Motif_f3[d][m]/sum(Motif_f3[d].values())  